# RNN

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from SR import *
from zigzag import *
import matplotlib.pyplot as plt
import mplfinance as mpf
from random import sample
import ta
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
cd data

/Users/kush/Desktop/DS/inpredo-master/RNN_tabular/AI-for-Trading/data


# Load Data


MAKE ALL THE DATA

In [61]:
files = sorted([i for i in os.listdir() if i[:3] == "DAT"])

df = pd.DataFrame(columns=["open", "high", "low", "close", "volume"])
for i in files:
    temp = pd.read_csv(i, names=["open", "high", "low", "close", "volume"])
    df = pd.concat([df, temp])

print('import done')
# fix index
df.index = pd.to_datetime([df.index[i][0] + " " + df.index[i][1] for i, _ in enumerate(df.index)])
print("index done")

# # change timeframe
logic = {'open'  : 'first',
         'high'  : 'max',
         'low'   : 'min',
         'close' : 'last',
         'volume': 'sum'}
offset = pd.offsets.timedelta(days=-6)

#df = pd.read_clipboard(parse_dates=['Date'], index_col=['Date'])
df = df.resample('15T', loffset=offset).apply(logic)
print("resample done")
# add all technical analysis features and get singals
#ta.add_momentum_ta(df, "high", "low", "close", "volume", fillna=True)
#ta.add_others_ta(df, "close", fillna=True)
#ta.add_trend_ta(df, "high", "low", "close", fillna=True)
#ta.add_volatility_ta(df, "high", "low", "close", fillna=True)
print("ta done")
zigzag(df)[0].to_csv("eurusd_15.csv")
print("zigzag done")

import done
index done
resample done
zigzag done
fib done


In [50]:
data[list(signals)]

,index,open,high,low,close,volume,momentum_rsi,momentum_tsi,momentum_uo,momentum_stoch,...,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,1,fib
156,2010-12-29 08:00:00,1.33920,1.34230,1.33920,1.34210,0,67.591296,22.870266,53.691828,95.652174,...,1.341000,1.337860,0.234428,1.350318,1.0,0.0,0.679876,0.978022,-1.0,1.016396
672,2011-01-03 17:00:00,1.28760,1.28790,1.28740,1.28760,0,37.006336,-26.824358,0.000000,40.000000,...,1.288133,1.287133,0.077662,0.466667,0.0,0.0,0.038832,0.400000,1.0,0.520209
1098,2011-01-08 03:30:00,1.34320,1.34490,1.34310,1.34400,0,83.519712,45.700038,57.402597,87.401575,...,1.340547,1.335907,0.346728,1.744253,1.0,0.0,0.965450,0.875969,-1.0,2.125000
1450,2011-01-11 19:30:00,1.32610,1.32620,1.32520,1.32570,0,29.666706,-22.196879,38.528147,11.363636,...,1.328480,1.326500,0.149154,-0.404040,0.0,1.0,0.331293,0.113636,1.0,0.494845
2374,2011-01-21 10:30:00,1.37350,1.37510,1.37270,1.37490,0,62.175237,15.291225,53.596132,83.673469,...,1.375363,1.371243,0.300007,0.887540,0.0,0.0,0.356898,0.836735,-1.0,1.079137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339713,2020-09-04 09:15:00,1.18997,1.19068,1.18912,1.19024,0,81.270628,45.069853,58.948524,94.195251,...,1.187967,1.184107,0.325453,1.588774,1.0,0.0,0.657485,0.943517,-1.0,0.896488
340345,2020-09-10 23:15:00,1.17624,1.17629,1.17373,1.17490,0,23.011390,-43.855835,41.350003,17.462687,...,1.177983,1.175903,0.176729,-0.482051,0.0,1.0,0.692473,0.143382,1.0,18.564706
340732,2020-09-15 00:00:00,1.18671,1.18717,1.18671,1.18706,0,76.823312,54.576662,61.524230,93.604651,...,1.186589,1.185767,0.069298,1.572587,1.0,0.0,0.243724,0.961938,-1.0,0.911839
341160,2020-09-19 11:00:00,1.16188,1.16233,1.16121,1.16139,0,33.506582,-28.772139,45.890142,5.642633,...,1.164116,1.161570,0.218946,-0.070568,0.0,1.0,0.348142,0.044444,1.0,-2.261404


# Testing Cells

In [ ]:
%time
lookback = 432
n_candles = 50

# get signals (as a column in df) and zigzag lines
df_zz, zz = zigzag(df, 1, 10)

df_sr = sr(df_zz,lookback)

# use this df to only show the last few candles
df_sr_image = df_sr[-n_candles:].copy()

# create a subplot for each level
apdict = [mpf.make_addplot(df_sr_image[c], color="w", width=5, alpha=0.5) for c in df_sr_image.columns[8:] if (c > df_sr_image[-n_candles:].low.min()*0.995 and int(c) < df_sr_image[-n_candles:].high.max()*1.005)]

# custom colors (mainly to make the background black)
# Create my own `marketcolors` to use with the `nightclouds` style:
mc = mpf.make_marketcolors(up='g',down='r',inherit=True)

# Create a new style based on `nightclouds` but with my own `marketcolors`:
s  = mpf.make_mpf_style(base_mpf_style='nightclouds',marketcolors=mc)

# plot level
mpf.plot(df_sr_image, type="candle", style=s, axisoff=True, addplot=apdict, mav=25, volume=True, figsize=(10,10))